In [ ]:
%%html
<link rel="import" href="urth_components/urth-viz-table/urth-viz-table.html" is="urth-core-import">
<link rel="import" href="urth_components/urth-viz-bar/urth-viz-bar.html" is="urth-core-import">
<link rel="import" href="urth_components/urth-viz-explorer/urth-viz-explorer.html" is="urth-core-import">
<link rel="import" href="urth_components/paper-input/paper-input.html" is='urth-core-import' package='PolymerElements/paper-input'>
<link rel="import" href="urth_components/paper-spinner/paper-spinner.html" is='urth-core-import' package='PolymerElements/paper-spinner'>

#### Pandas DataFrame

In [ ]:
import pandas as pd

In [ ]:
fares = pd.read_csv('../demos/tour/fares.csv')
fares.columns = fares.columns.map(lambda x: x.strip())
fares.columns

In [ ]:
%%html
<template id="t" is="dom-bind">
    <urth-core-dataframe id="d" ref="fares" value="{{df}}">
        <urth-core-query-filter disabled="{{!vid}}">vendor_id == '{{vid}}'</urth-core-query-filter>
        <urth-core-query-filter disabled="{{!ptype}}">payment_type == '{{ptype}}'</urth-core-query-filter>
        <urth-core-query-group by="{{gCol}}">
            <urth-core-query-agg op="sum" col="fare_amount"></urth-core-query-agg>
            <urth-core-query-agg op="mean" col="fare_amount"></urth-core-query-agg>
            <urth-core-query-agg op="mean" col="tolls_amount"></urth-core-query-agg>
            <urth-core-query-agg op="sum" col="tolls_amount"></urth-core-query-agg>
        </urth-core-query-group>
    </urth-core-dataframe>
    <paper-input label="By" value="{{gCol}}"></paper-input>
    <paper-input label="Vendor ID" value="{{vid}}"></paper-input>
    <paper-input label="Payment Type" value="{{ptype}}"></paper-input>
    <urth-viz-table datarows="{{df.data}}" columns="{{df.columns}}"></urth-viz-table>
    <urth-viz-bar datarows="{{df.data}}"></urth-viz-bar>
</template>

#### PySpark DataFrame

In [ ]:
import pyspark
sc = pyspark.SparkContext('local[*]')
sqlContext = pyspark.SQLContext(sc)

In [ ]:
spark_fares = sqlContext.createDataFrame(fares)

In [ ]:
%%html
<template id="t" is="dom-bind">
    <urth-core-dataframe id="d" ref="spark_fares" value="{{df}}" busy="{{busy}}">
        <urth-core-query-filter disabled="{{!vid}}">vendor_id = '{{vid}}'</urth-core-query-filter>
        <urth-core-query-filter disabled="{{!ptype}}">payment_type = '{{ptype}}'</urth-core-query-filter>
        <urth-core-query-group by="{{gCol}}">
            <urth-core-query-agg op="sum" col="fare_amount"></urth-core-query-agg>
            <urth-core-query-agg op="mean" col="fare_amount"></urth-core-query-agg>
            <urth-core-query-agg op="mean" col="tolls_amount"></urth-core-query-agg>
            <urth-core-query-agg op="sum" col="tolls_amount"></urth-core-query-agg>
        </urth-core-query-group>
    </urth-core-dataframe>
    <paper-input label="By" value="{{gCol}}"></paper-input>
    <paper-input label="Vendor ID" value="{{vid}}"></paper-input>
    <paper-input label="Payment Type" value="{{ptype}}"></paper-input>
    <paper-spinner active="{{busy}}"></paper-spinner>
    <urth-viz-table datarows="{{df.data}}" columns="{{df.columns}}"></urth-viz-table>
    <urth-viz-bar datarows="{{df.data}}"></urth-viz-bar>
</template>

#### Using urth-viz-explorer

In [ ]:
%%html
<template is="dom-bind">
    <paper-input label="By" value="{{gCol}}"></paper-input>
    <paper-input label="Vendor ID" value="{{vid}}"></paper-input>
    <paper-input label="Payment Type" value="{{ptype}}"></paper-input>
    <paper-spinner style="float:right;" active="{{busy}}"></paper-spinner>
    <urth-viz-explorer>
        <urth-core-dataframe id="d" ref="fares" busy="{{busy}}">
            <urth-core-query-filter disabled="{{!vid}}">vendor_id == '{{vid}}'</urth-core-query-filter>
            <urth-core-query-filter disabled="{{!ptype}}">payment_type == '{{ptype}}'</urth-core-query-filter>
            <urth-core-query-group by="{{gCol}}">
                <urth-core-query-agg op="sum" col="fare_amount"></urth-core-query-agg>
                <urth-core-query-agg op="mean" col="fare_amount"></urth-core-query-agg>
                <urth-core-query-agg op="mean" col="tolls_amount"></urth-core-query-agg>
                <urth-core-query-agg op="sum" col="tolls_amount"></urth-core-query-agg>
            </urth-core-query-group>
        </urth-core-dataframe>
    </urth-viz-explorer>
</template>

#### Taxi demo reworked

In [ ]:
%%html
<link rel="import" href="urth_components/paper-dropdown-menu/paper-dropdown-menu.html" is='urth-core-import' package='PolymerElements/paper-dropdown-menu'>
<link rel="import" href="urth_components/paper-menu/paper-menu.html" is='urth-core-import' package='PolymerElements/paper-menu'>
<link rel="import" href="urth_components/paper-item/paper-item.html" is='urth-core-import' package='PolymerElements/paper-item'>
<link rel="import" href="urth_components/paper-button/paper-button.html" is='urth-core-import' package='PolymerElements/paper-button'>
<link rel="import" href="urth_components/paper-card/paper-card.html" is='urth-core-import' package='PolymerElements/paper-card'>
<link rel="import" href="urth_components/paper-slider/paper-slider.html" is='urth-core-import' package='PolymerElements/paper-slider'>

<style is="custom-style">
    urth-viz-chart::shadow .nv-x .tick text {
        display: none;
    }
</style>

In [ ]:
from urth.widgets.widget_channels import channel
channel("faresChan").set("paymentOptions", [["All", ""], ["Cash", "CSH"], ["Credit","CRD"]])

In [ ]:
%%html
<template id="earnings" is="urth-core-bind" channel="faresChan">
    <urth-core-dataframe id="df" ref="fares" value="{{top}}" limit="{{limit}}" busy="{{busy}}">
        <urth-core-query-filter disabled="{{!payType}}">payment_type == '{{payType}}'</urth-core-query-filter>
        <urth-core-query-group by="medallion">
            <urth-core-query-agg op="sum" col="total_amount"></urth-core-query-agg>
        </urth-core-query-group>
        <urth-core-query-sort by="total_amount_sum" direction="{{topOrBottom}}"></urth-core-query-sort>
    </urth-core-dataframe>
    <paper-card style="width: 100%;" heading="Earners" elevation="1">
        <div class="card-content">
            <paper-spinner style="float:right;" active="{{busy}}"></paper-spinner>
            <paper-dropdown-menu label="Select Payment Type" noink>
                <paper-menu class="dropdown-content" selected="{{payType}}" attr-for-selected="value">
                    <template is="dom-repeat" items="[[paymentOptions]]">
                        <paper-item value="[[item.1]]">[[item.0]]</paper-item>
                    </template>
                </paper-menu>
            </paper-dropdown-menu>
            <paper-dropdown-menu label="Top or Bottom Earners" noink>
                <paper-menu class="dropdown-content" selected="{{topOrBottom}}" attr-for-selected="value">
                    <paper-item value="desc">Top</paper-item>
                    <paper-item value="asc">Bottom</paper-item>
                </paper-menu>
            </paper-dropdown-menu>

            <urth-viz-chart type='bar' datarows='{{top.data}}' columns='{{top.columns}}' selection="{{sel}}" multi-select></urth-viz-chart>
            <div class="center horizontal layout">
                <div>Limit</div>
                <paper-slider min="5" max="50" step="5" pin snap value="{{limit}}"></paper-slider>
            </div>
        </div>
    </paper-card>
</template>